In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'e:\\EndToEnd\\01_MLOps'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/deasadiqbal/01_MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="deasadiqbal"
os.environ["MLFLOW_TRACKING_PASSWORD"]="56b8918094b5da7d4d487c082b35b97b710ede54"

## Entity

In [51]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_dir: Path
    model_dir: Path
    metrix_filename: Path
    all_params: dict
    target_col: str
    mlflow_uri: str

## Configratution


In [57]:
from mlops_wine_quality.utils.common import read_yaml, create_dirs
from mlops_wine_quality.constants import *

class ConfigManger:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        target_col = self.schema.TARGET_COLUMN
        params = self.params.Elasticnet

        create_dirs([config.root_dir])

        configs = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_dir=config.test_data_dir,
            model_dir=config.model_dir,
            metrix_filename=config.metrix_filename,
            all_params=params,                        
            target_col=target_col.name,
            mlflow_uri="https://dagshub.com/deasadiqbal/01_MLOps.mlflow"
        )
        return configs

## components

In [58]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlops_wine_quality.utils.common import save_json
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evel_metrix(self, y_test, predicitons):
        rmse = np.sqrt(mean_squared_error(y_test, predicitons))
        mae = mean_absolute_error(y_test, predicitons)
        r2 = r2_score(y_test, predicitons)
        return rmse, mae, r2

    def start_with_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_dir)
        model_dir = self.config.model_dir

        lr_model = joblib.load(model_dir)

        x_test = test_data.drop(self.config.target_col, axis=1) 
        y_test = test_data[self.config.target_col]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            perdictions = lr_model.predict(x_test)
            (rmse, mae, r2) = self.evel_metrix(y_test, perdictions)

            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path=Path(self.config.metrix_filename), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(lr_model, "model", registered_model_name="ElasticnetWineModel")
            else:
                mlflow.sklearn.log_model(lr_model, "model")

## Pipeline

In [59]:
try:
    config = ConfigManger()
    model_evaluation_config = config.get_model_evaluation_config()

    model_eval = ModelEvaluation(model_evaluation_config)
    model_eval.start_with_mlflow()
except Exception as e:
    raise e

[2024-01-21 18:36:05,169: INFO: common: Yaml file loaded successfully]
[2024-01-21 18:36:05,177: INFO: common: Yaml file loaded successfully]
[2024-01-21 18:36:05,184: INFO: common: Yaml file loaded successfully]
[2024-01-21 18:36:05,186: INFO: common: Created directory at artifacts/model_evaluation]


[2024-01-21 18:36:05,941: INFO: common: Saved json file at artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetWineModel' already exists. Creating a new version of this model...
2024/01/21 18:36:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 2
Created version '2' of model 'ElasticnetWineModel'.
